In [58]:

# import required module
import os
import pandas as pd
import numpy as np

import country_converter as coco
cc = coco.CountryConverter()

In [59]:
##Add GDP per capita before removing years
GDPpc_Data = (pd.read_excel("../mpd2020.xlsx", sheet_name = "Full data")
                .drop(columns = ["pop", "countrycode"])
                .rename(columns = {"gdppc" : "GDPPC", "country" : "Country", "year" : "Year"})
                .replace({'United States' : 'USA'}))
#GDPpc_Data.head()
GDPpc_Data["Country"] = cc.convert(names = GDPpc_Data['Country'], to = "name_short")
GDPpc_Data.GDPPC =  pd.to_numeric(GDPpc_Data.GDPPC)  #Convert column to numeric for calculations

D.R. of the Congo not found in regex
D.R. of the Congo not found in regex
D.R. of the Congo not found in regex
D.R. of the Congo not found in regex
D.R. of the Congo not found in regex
D.R. of the Congo not found in regex
D.R. of the Congo not found in regex
D.R. of the Congo not found in regex
D.R. of the Congo not found in regex
D.R. of the Congo not found in regex
D.R. of the Congo not found in regex
D.R. of the Congo not found in regex
D.R. of the Congo not found in regex
D.R. of the Congo not found in regex
D.R. of the Congo not found in regex
D.R. of the Congo not found in regex
D.R. of the Congo not found in regex
D.R. of the Congo not found in regex
D.R. of the Congo not found in regex
D.R. of the Congo not found in regex
D.R. of the Congo not found in regex
D.R. of the Congo not found in regex
D.R. of the Congo not found in regex
D.R. of the Congo not found in regex
D.R. of the Congo not found in regex
D.R. of the Congo not found in regex
D.R. of the Congo not found in regex
D

In [60]:
GDPpc_Data

,Country,Year,GDPPC
0,Afghanistan,1820,NaN
1,Afghanistan,1870,NaN
2,Afghanistan,1913,NaN
3,Afghanistan,1950,1156.0000
4,Afghanistan,1951,1170.0000
...,...,...,...
21677,Zimbabwe,2014,1594.0000
21678,Zimbabwe,2015,1560.0000
21679,Zimbabwe,2016,1534.0000
21680,Zimbabwe,2017,1582.3662


In [61]:
TWWI = pd.DataFrame(columns=['Country', 'Year', 'TWWI'])

# assign directory
directory = 'DOTdata'
 
# iterate over files in
# that directory
for filename in os.listdir(directory):
#for i in range(1,187):
    #f = f"DOTdata\Exports_to_Count ({i}).xlsx"
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        Coun = ( pd.read_excel(f, skiprows = 3, nrows = 0, usecols = 'B')
        .columns[0] )
        Coun = cc.convert(names = Coun, to = "name_short")
        print(f)

DOTdata\Exports_to_Count (1).xlsx
DOTdata\Exports_to_Count (10).xlsx
DOTdata\Exports_to_Count (100).xlsx
DOTdata\Exports_to_Count (101).xlsx
DOTdata\Exports_to_Count (102).xlsx
DOTdata\Exports_to_Count (103).xlsx
DOTdata\Exports_to_Count (104).xlsx
DOTdata\Exports_to_Count (105).xlsx
DOTdata\Exports_to_Count (107).xlsx
DOTdata\Exports_to_Count (108).xlsx
DOTdata\Exports_to_Count (109).xlsx
DOTdata\Exports_to_Count (11).xlsx
DOTdata\Exports_to_Count (110).xlsx
DOTdata\Exports_to_Count (111).xlsx
DOTdata\Exports_to_Count (112).xlsx
DOTdata\Exports_to_Count (113).xlsx
DOTdata\Exports_to_Count (114).xlsx
DOTdata\Exports_to_Count (115).xlsx
DOTdata\Exports_to_Count (116).xlsx
DOTdata\Exports_to_Count (117).xlsx
DOTdata\Exports_to_Count (118).xlsx
DOTdata\Exports_to_Count (119).xlsx
DOTdata\Exports_to_Count (12).xlsx
DOTdata\Exports_to_Count (120).xlsx
DOTdata\Exports_to_Count (121).xlsx
DOTdata\Exports_to_Count (122).xlsx
DOTdata\Exports_to_Count (123).xlsx
DOTdata\Exports_to_Count (124).xl

Eastern Germany not found in regex


DOTdata\Exports_to_Count (150).xlsx
DOTdata\Exports_to_Count (151).xlsx
DOTdata\Exports_to_Count (152).xlsx
DOTdata\Exports_to_Count (153).xlsx
DOTdata\Exports_to_Count (154).xlsx
DOTdata\Exports_to_Count (155).xlsx
DOTdata\Exports_to_Count (156).xlsx
DOTdata\Exports_to_Count (157).xlsx
DOTdata\Exports_to_Count (158).xlsx
DOTdata\Exports_to_Count (159).xlsx
DOTdata\Exports_to_Count (16).xlsx
DOTdata\Exports_to_Count (160).xlsx
DOTdata\Exports_to_Count (161).xlsx
DOTdata\Exports_to_Count (162).xlsx
DOTdata\Exports_to_Count (163).xlsx
DOTdata\Exports_to_Count (164).xlsx
DOTdata\Exports_to_Count (165).xlsx
DOTdata\Exports_to_Count (166).xlsx
DOTdata\Exports_to_Count (167).xlsx
DOTdata\Exports_to_Count (168).xlsx
DOTdata\Exports_to_Count (169).xlsx
DOTdata\Exports_to_Count (17).xlsx
DOTdata\Exports_to_Count (170).xlsx
DOTdata\Exports_to_Count (171).xlsx
DOTdata\Exports_to_Count (172).xlsx
DOTdata\Exports_to_Count (173).xlsx
DOTdata\Exports_to_Count (174).xlsx
DOTdata\Exports_to_Count (175)

Netherlands Antilles not found in regex


DOTdata\Exports_to_Count (37).xlsx
DOTdata\Exports_to_Count (38).xlsx
DOTdata\Exports_to_Count (39).xlsx
DOTdata\Exports_to_Count (4).xlsx
DOTdata\Exports_to_Count (40).xlsx
DOTdata\Exports_to_Count (41).xlsx
DOTdata\Exports_to_Count (42).xlsx
DOTdata\Exports_to_Count (43).xlsx
DOTdata\Exports_to_Count (44).xlsx
DOTdata\Exports_to_Count (45).xlsx
DOTdata\Exports_to_Count (46).xlsx
DOTdata\Exports_to_Count (47).xlsx
DOTdata\Exports_to_Count (48).xlsx
DOTdata\Exports_to_Count (49).xlsx
DOTdata\Exports_to_Count (5).xlsx
DOTdata\Exports_to_Count (50).xlsx
DOTdata\Exports_to_Count (51).xlsx
DOTdata\Exports_to_Count (52).xlsx
DOTdata\Exports_to_Count (53).xlsx
DOTdata\Exports_to_Count (54).xlsx
DOTdata\Exports_to_Count (55).xlsx
DOTdata\Exports_to_Count (56).xlsx
DOTdata\Exports_to_Count (57).xlsx
DOTdata\Exports_to_Count (58).xlsx
DOTdata\Exports_to_Count (59).xlsx
DOTdata\Exports_to_Count (6).xlsx
DOTdata\Exports_to_Count (60).xlsx
DOTdata\Exports_to_Count (61).xlsx


...

DOTdata\Exports_to_Count (62).xlsx
DOTdata\Exports_to_Count (63).xlsx
DOTdata\Exports_to_Count (64).xlsx
DOTdata\Exports_to_Count (65).xlsx
DOTdata\Exports_to_Count (66).xlsx
DOTdata\Exports_to_Count (67).xlsx
DOTdata\Exports_to_Count (68).xlsx
DOTdata\Exports_to_Count (69).xlsx
DOTdata\Exports_to_Count (7).xlsx
DOTdata\Exports_to_Count (70).xlsx
DOTdata\Exports_to_Count (71).xlsx
DOTdata\Exports_to_Count (72).xlsx
DOTdata\Exports_to_Count (73).xlsx
DOTdata\Exports_to_Count (74).xlsx
DOTdata\Exports_to_Count (75).xlsx
DOTdata\Exports_to_Count (76).xlsx
DOTdata\Exports_to_Count (77).xlsx
DOTdata\Exports_to_Count (78).xlsx
DOTdata\Exports_to_Count (79).xlsx
DOTdata\Exports_to_Count (8).xlsx
DOTdata\Exports_to_Count (80).xlsx
DOTdata\Exports_to_Count (81).xlsx
DOTdata\Exports_to_Count (82).xlsx


USSR not found in regex


DOTdata\Exports_to_Count (83).xlsx
DOTdata\Exports_to_Count (84).xlsx
DOTdata\Exports_to_Count (85).xlsx
DOTdata\Exports_to_Count (86).xlsx
DOTdata\Exports_to_Count (87).xlsx
DOTdata\Exports_to_Count (88).xlsx
DOTdata\Exports_to_Count (89).xlsx
DOTdata\Exports_to_Count (9).xlsx
DOTdata\Exports_to_Count (90).xlsx
DOTdata\Exports_to_Count (91).xlsx


Yugoslavia not found in regex


DOTdata\Exports_to_Count (92).xlsx
DOTdata\Exports_to_Count (93).xlsx
DOTdata\Exports_to_Count (94).xlsx
DOTdata\Exports_to_Count (95).xlsx
DOTdata\Exports_to_Count (96).xlsx
DOTdata\Exports_to_Count (97).xlsx
DOTdata\Exports_to_Count (98).xlsx
DOTdata\Exports_to_Count (99).xlsx


In [141]:
f = "DOTdata\Exports_to_Count (99).xlsx"

Coun = ( pd.read_excel(f, skiprows = 3, nrows = 0, usecols = 'B')
        .columns[0] ) #gets country name only
Coun = cc.convert(names = Coun, to = "name_short")

temp = ( pd.read_excel(f, skiprows = 6)
        .drop(columns = ["Unnamed: 0"])
        .rename(columns= {'Unnamed: 1': 'Country'})
        .fillna(0) ) #gets data

#Coun
temp = temp[temp['Country']!= "Aruba, Kingdom of the Netherlands"]
temp = temp[temp['Country']!= "Curaçao, Kingdom of the Netherlands"] #countries that don't matter
temp = temp[temp['Country']!= "Sint Maarten, Kingdom of the Netherlands"]

temp.Country = cc.convert(names = temp['Country'], to = "name_short")
temp = temp[temp["Country"] != "not found"]
temp = temp.melt(id_vars='Country', var_name="Year", value_name="ExpRec")
temp.Year = temp.Year.astype(int)
temp.ExpRec = temp.apply(lambda x: float(x['ExpRec'].split(" ")[0].replace(',','')) if type(x['ExpRec']) == str else x['ExpRec'], axis=1) ## 1.2 e issue solving

CounGDP = GDPpc_Data[GDPpc_Data.Country == Coun]
temp2 = temp.merge(CounGDP[['Year', 'GDPPC']], on ='Year', how = 'left').rename(columns = {'GDPPC': 'DomGDPPC'})
temp2['w'] =  temp2.apply(lambda x: 0 if np.isnan(x['DomGDPPC']) else x['ExpRec'] / x['DomGDPPC'] * 1e6 , axis=1)  ##calculate w per year
temp2 = temp2.sort_values(by = ["Country", "Year"])

TW = ( temp2.groupby('Country')[["Year","w"]]
      .rolling(5, min_periods=5, on='Year', closed = "left")
      .mean().reset_index().drop(columns = "level_1")
      .rename(columns = {'w':'TW'}).fillna(0) ) ##calculating TW

temp3 = temp2.merge(TW, on= ['Year','Country'], how = "outer").merge(GDPpc_Data, on = ["Country", "Year"], how = "left")
temp3['TWi'] = temp3.apply(lambda x: x['TW'] * x['GDPPC'], axis = 1)
#temp3.tail(50)

temp4 = temp3.groupby('Year')['TWi'].sum().reset_index().rename(columns = {"TWi": 'TWWI'})
temp4['Country'] = Coun
temp4

Belgium-Luxembourg not found in regex
Eastern Germany not found in regex
Netherlands Antilles not found in regex
Serbia and Montenegro not found in regex
Yugoslavia not found in regex


,Year,TWWI,Country
0,1961,0.000000e+00,Algeria
1,1966,0.000000e+00,Algeria
2,1967,1.106269e+06,Algeria
3,1968,5.857015e+06,Algeria
4,1969,1.277316e+07,Algeria
5,1970,3.658725e+09,Algeria
6,1971,4.176730e+09,Algeria
7,1972,4.258084e+09,Algeria
8,1973,4.645820e+09,Algeria
9,1974,5.652436e+09,Algeria
